In [19]:
import pandas as pd
import numpy as np
import os
import time
from rdkit import Chem
from fp_gen import KlekFPGenerator, MACCSFPGenerator, SubFPGenerator

In [20]:
generator = KlekFPGenerator(n_jobs=16)
data = pd.read_csv('../GRU_data/250k_zinc.csv', chunksize=5000, names=['smiles'])

In [21]:
def could_be_valid(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

In [22]:
def sparse_to_dense(sparse):
    return np.nonzero(sparse)[0].tolist()

In [23]:
for i, chunk in enumerate(data):
    print(i)
    chunk = chunk[chunk['smiles'].apply(could_be_valid)]
    mols = list(map(Chem.MolFromSmiles, chunk['smiles']))
    fps = generator.transform(mols)
    chunk['fps'] = list(map(sparse_to_dense, fps))
    if i == 0:
        chunk.to_csv('zinc_klek.csv', index=False)
    with open('zinc_klek.csv', 'a') as f:
        chunk.to_csv(f, header=False, index=False)

0


[19:03:36] SMILES Parse Error: syntax error while parsing: SAS
[19:03:36] SMILES Parse Error: Failed parsing SMILES 'SAS' for input: 'SAS'
[19:03:36] SMILES Parse Error: syntax error while parsing: 2.0840945720726807
[19:03:36] SMILES Parse Error: Failed parsing SMILES '2.0840945720726807' for input: '2.0840945720726807'
[19:03:36] SMILES Parse Error: syntax error while parsing: 3.4320038192747795
[19:03:36] SMILES Parse Error: Failed parsing SMILES '3.4320038192747795' for input: '3.4320038192747795'
[19:03:36] SMILES Parse Error: syntax error while parsing: 2.4706326078252
[19:03:36] SMILES Parse Error: Failed parsing SMILES '2.4706326078252' for input: '2.4706326078252'
[19:03:36] SMILES Parse Error: syntax error while parsing: 2.8227533111709646
[19:03:36] SMILES Parse Error: Failed parsing SMILES '2.8227533111709646' for input: '2.8227533111709646'
[19:03:36] SMILES Parse Error: syntax error while parsing: 4.035182138302743
[19:03:36] SMILES Parse Error: Failed parsing SMILES '4.0

1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: 

In [18]:
reload_data = pd.read_csv('zinc_klek.csv')
reload_data['fps'] = reload_data['fps'].apply(eval)
reload_data.head()

,smiles,fps
